In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import patsy
from pivot_table_lm.nodes.cross_validate import compare_models

pd.set_option('display.max_columns', None)

%reload_kedro
%load_ext autoreload
%autoreload 2

In [ ]:
df = io.load("clean_automobile")
parameters = io.load("parameters")

In [ ]:
# df.groupby(["make", "num_cylinders"])["price"].mean().dropna().sort_values()

In [ ]:
# mod = smf.ols("price ~ make:num_cylinders - 1", df)
# res = mod.fit()
# res.summary2().tables[1].dropna().round(1).sort_values("Coef.")

In [ ]:
result = compare_models(df, parameters)

In [ ]:
result

In [ ]:
df = df.dropna()

In [ ]:
y, X = patsy.dmatrices("price ~ 1", df, return_type="dataframe")
X.shape

In [ ]:
reg = LinearRegression(fit_intercept=False)
scores = cross_validate(reg, X, y, cv=5, scoring=("r2"))
base = scores['test_score'].mean()

In [ ]:
y, X = patsy.dmatrices("price ~ make", df, return_type="dataframe")
reg = LinearRegression(fit_intercept=False)
scores = cross_validate(reg, X, y, cv=5, scoring=("r2"))
conditional_mean = scores['test_score'].mean()
X.shape

In [ ]:
all_cols = "+".join([col for col in df.drop("price",axis=1).columns])
y, X = patsy.dmatrices("price ~ " + all_cols, df, return_type="dataframe")
X.shape

In [ ]:
reg = RandomForestRegressor(n_estimators=500)
scores = cross_validate(reg, X.values, y.values.ravel(), cv=5, scoring=("r2"))
rf = scores['test_score'].mean()

In [ ]:
base

In [ ]:
rf

In [ ]:
conditional_mean